# <span style = 'color:green'> Descubriendo la cinemática con Machine Learning
Veamos un ejemplo bastante simple: la <span style = 'color:#eb3433'> fuerza de gravedad </span> sobre un objeto en caída. La ecuación que define el <span style = 'color:#eb3433'> movimiento de esta partícula </span> es:
    $$y = x_{0} + v_{0}t - \frac{1}{2} gt$$
    
**Creación del set de datos**

Creamos una función en Python para calcular esta posición.

In [1]:
import csv
import random
import math
random.seed
import pandas as pd


#\usepackage[shortconst]{physconst}

In [2]:
def location(x_0, v_0, t):
    return x_0 + v_0*t - (9.82/2)*t**2

Creo un dataset para guardarlo como un archivo CSV.


In [3]:
with open('gravity_location_data.csv', mode='w') as gravity_file:
    gravity_writer = csv.writer(gravity_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    gravity_writer.writerow(['initial_position', 'initial_velocity', 'mass', 'time', 'location'])
    
    for i in range (0, 10000):
        initial_position = random.randrange(1, 10000)
        initial_velocity = random.randrange(1, 100)
        mass = random.randrange(1, 1000)
        time = random.randrange(1, 100)
        gravity_writer.writerow([initial_position, initial_velocity, mass, time, location(initial_position, initial_velocity, time)])
   

¿por qué <span style = 'color:#eb3433'> incluir la masa </span> en este conjunto de datos? Porque queremos intentar engañar al algoritmo de aprendizaje de la máquina para que lo use.

Importemos el nuevo archivo de datos...

In [4]:
gravity_data = pd.read_csv('gravity_location_data.csv')
df_location = pd.DataFrame(gravity_data)

Como es habitual en el aprendizaje automático, necesitaremos dividir nuestros datos en un <span style = 'color:#eb3433'> conjunto de entrenamiento </span>  (para entrenar el modelo) y un <span style = 'color:#eb3433'> conjunto de pruebas </span> (para evaluar el modelo entrenado). Para ellos se usa una función especial para Python: sklearn

In [5]:
from sklearn.model_selection import train_test_split
def split_data(data, target_name):
    y = data[target_name]
    X = data.drop(target_name, axis=1)
    return train_test_split(X, y, test_size=0.2, random_state=30)

Para entrenar y evaluar nuestro modelo, usaremos la siguiente función:

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt
def train_eval_poly(X_train, X_test, y_train, y_test):
    regression_model = LinearRegression() 
    
    poly = PolynomialFeatures(degree=2)
    X_train_transform = poly.fit_transform(X_train)
    X_test_transform = poly.fit_transform(X_test)
    
    regression_model.fit(X_train_transform, y_train)
    
    print(poly.fit(X_train).get_feature_names(X_train.columns))
    
    y_pred = regression_model.predict(X_test_transform)
    print("R2: \t", r2_score(y_test, y_pred))
    print("RMSE: \t", sqrt(mean_squared_error(y_test, y_pred)))
    print("MAE: \t", mean_absolute_error(y_test, y_pred))
    
    return regression_model

Nótese que esto es sólo una Regresión Lineal, excepto que usamos la función los Polinomios Característicos para transformar los datos de manera que la Regresión Lineal pueda modelar correctamente un polinomio de segundo grado, lo cual sabemos que en este caso es exactamente lo que estamos buscando. A continuación veremos un poco más sobre cómo se transforman los datos.

Después de entrenar el modelo, obtendremos los nombres de las características, que nos dirán cómo se asignan los coeficientes resultantes.
Ahora entrenamos el modelo con este código:


In [7]:
df_split = split_data(df_location, 'location')
lrModel = train_eval_poly(*df_split)

['1', 'initial_position', 'initial_velocity', 'mass', 'time', 'initial_position^2', 'initial_position initial_velocity', 'initial_position mass', 'initial_position time', 'initial_velocity^2', 'initial_velocity mass', 'initial_velocity time', 'mass^2', 'mass time', 'time^2']
R2: 	 1.0
RMSE: 	 2.522726153057001e-09
MAE: 	 2.1540236041506234e-09


Tenga en cuenta que el valor de r² es 1.0 -  Además, la raíz media del error cuadrado (RMSE) y la media del error absoluto (MAE) son minúsculas - el e-09 significa multiplicar el número precedente por .000000001. Así que la raíz media del error al cuadrado es en realidad 0,00000000285... A esta escala eso es minúsculo, esencialmente cero.

Observe también la lista inicial de 15 elementos en los resultados anteriores. Este es el resultado del método PolynomialFeatures. Transformó el conjunto inicial de cuatro características ('posición_inicial', 'velocidad_inicial', 'masa', 'tiempo') y las cambió en combinaciones de estas características, como 'posición_inicial ^ 2' y 'tiempo_de_posición_inicial', cuyos valores son posición_inicial multiplicada por el tiempo. Este conjunto expandido de características fue entonces alimentado en la simple y vieja Regresión Lineal. Este es un pequeño truco que nos permite realizar la regresión polinómica no con un algoritmo diferente sino con datos transformados.
Ahora veamos los exponentes de nuestro modelo, que son los exponentes de la ecuación polinómica que el modelo está usando para hacer predicciones:



In [8]:
lrModel.coef_


array([ 0.00000000e+00,  1.00000000e+00, -5.09744168e-13, -1.07288336e-13,
       -3.41389880e-13, -1.11022302e-16,  2.58256957e-16,  1.66533454e-16,
       -5.86336535e-16,  7.30679081e-15, -1.05232663e-15,  1.00000000e+00,
       -5.72675588e-16, -1.47803861e-15, -4.91000000e+00])

El primer valor (0.00000000e+00, o 0) es el coeficiente para el valor 1. Eso sería una constante colgante sin ninguna variable, un valor constante siempre añadido a la ecuación, excepto que el coeficiente es cero por lo que no hay un valor constante añadido a nuestro resultado. El segundo valor de nuestra matriz ( 1.00000000e+00, o 1) es el coeficiente en "posición_inicial". Y de hecho, en nuestra ecuación original, tenemos la posición inicial (X₀) con un coeficiente de uno.

La mayoría de nuestros otros coeficientes son esencialmente cero, y nos dicen que estos factores son irrelevantes para predecir la ubicación, pero tenemos algunos que no lo son. El duodécimo de nuestra lista es también un coeficiente de 1, que corresponde al "tiempo_de_velocidad_inicial", o velocidad inicial multiplicada por el tiempo. De nuevo, tenemos este factor en nuestra ecuación original, que aparece como V₀*t.
Finalmente, el último valor es -4.9, que es el coeficiente correspondiente al 'tiempo ^ 2'. Note que esto también aparece en nuestra ecuación original como -1/2 gt². Pero dijimos antes que g es aproximadamente 9,8, y -1/2 * 9,8 = -4,9. Así que esto también se corresponde con nuestra ecuación original.

Si se comparan estos coeficientes con las combinaciones de variables de la lista de nombres de características, y se suman, e ignoran los casos en que el coeficiente es esencialmente cero, se terminaría con una ecuación muy cercana a la inicial.

Hay dos puntos finales muy interesantes para hacer aquí. Primero, el modelo resultante encontró que la masa era irrelevante para hacer la predicción, lo cual es correcto.

Segundo, el algoritmo de aprendizaje de la máquina fue capaz de derivar la constante de aceleración gravitatoria g, o al menos determinar el valor de todo el coeficiente (-1/2 * 9,8) que incluye g.

En otras palabras, parte del trabajo de Isaac Newton y Galileo puede ser descubierto en segundos con Python y scikit-learn. Esto es bastante emocionante.

Por supuesto, el análisis y la comprensión de estos resultados todavía requiere una importante intervención humana, pero esto al menos nos da una idea de cómo se puede extraer de los datos la comprensión de las leyes fundamentales del universo.
